In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV
import matplotlib.pylab as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.metrics import make_scorer
from sklearn.cross_validation import StratifiedShuffleSplit
from matplotlib.colors import LogNorm
from sklearn.decomposition import PCA
from keras.layers.advanced_activations import PReLU
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from keras.utils import np_utils
from copy import deepcopy
# %matplotlib inline  

/Library/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
Using Theano backend.


In [2]:
trainDF=pd.read_csv("../data/train.csv")

In [3]:
xy_scaler=preprocessing.StandardScaler()
print "abc"
xy_scaler.fit(trainDF[["X","Y"]])
print "abc"
trainDF[["X","Y"]]=xy_scaler.transform(trainDF[["X","Y"]])
print "abc"
trainDF=trainDF[abs(trainDF["Y"])<100]
print "abc"
trainDF.index=range(len(trainDF))
# plt.plot(trainDF["X"],trainDF["Y"],'.')
# plt.show()

abc
abc
abc
abc


In [4]:
NX=100
NY=100
groups = trainDF.groupby('Category')
ii=1
plt.figure(figsize=(20, 20))
for name, group in groups:
    plt.subplot(8,5,ii)
    histo, xedges, yedges = np.histogram2d(np.array(group.X),np.array(group.Y), bins=(NX,NY))
    myextent  =[xedges[0],xedges[-1],yedges[0],yedges[-1]]
    plt.imshow(histo.T,origin='low',extent=myextent,interpolation='nearest',aspect='auto',norm=LogNorm())
    plt.title(name)
#     plt.figure(ii)
#     plt.plot(group.X,group.Y,'.')
    ii+=1
del groups

In [5]:
def parse_time(x):
    DD=datetime.strptime(x,"%Y-%m-%d %H:%M:%S")
    time=DD.hour#*60+DD.minute
    day=DD.day
    month=DD.month
    year=DD.year
    return time,day,month,year

def get_season(x):
    summer=0
    fall=0
    winter=0
    spring=0
    if (x in [5, 6, 7]):
        summer=1
    if (x in [8, 9, 10]):
        fall=1
    if (x in [11, 0, 1]):
        winter=1
    if (x in [2, 3, 4]):
        spring=1
    return summer, fall, winter, spring

In [8]:
def parse_data(df,logodds,logoddsPA):
    feature_list=df.columns.tolist()
    if "Descript" in feature_list:
        feature_list.remove("Descript")
    if "Resolution" in feature_list:
        feature_list.remove("Resolution")
    if "Category" in feature_list:
        feature_list.remove("Category")
    if "Id" in feature_list:
        feature_list.remove("Id")
    cleanData=df[feature_list]
    cleanData.index=range(len(df))
    print "Creating address features"
    address_features=cleanData["Address"].apply(lambda x: logodds[x])
    address_features.columns=["logodds"+str(x) for x in range(len(address_features.columns))]
    print "Parsing dates"
    cleanData["Time"], cleanData["Day"], cleanData["Month"], cleanData["Year"]=zip(*cleanData["Dates"].apply(parse_time))
#     dummy_ranks_DAY = pd.get_dummies(cleanData['DayOfWeek'], prefix='DAY')
    days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
#     cleanData["DayOfWeek"]=cleanData["DayOfWeek"].apply(lambda x: days.index(x)/float(len(days)))
    print "Creating one-hot variables"
    dummy_ranks_PD = pd.get_dummies(cleanData['PdDistrict'], prefix='PD')
    dummy_ranks_DAY = pd.get_dummies(cleanData["DayOfWeek"], prefix='DAY')
    cleanData["IsInterection"]=cleanData["Address"].apply(lambda x: 1 if "/" in x else 0)
    cleanData["logoddsPA"]=cleanData["Address"].apply(lambda x: logoddsPA[x])
    print "droping processed columns"
    cleanData=cleanData.drop("PdDistrict",axis=1)
    cleanData=cleanData.drop("DayOfWeek",axis=1)
    cleanData=cleanData.drop("Address",axis=1)
    cleanData=cleanData.drop("Dates",axis=1)
    feature_list=cleanData.columns.tolist()
    print "joining one-hot features"
    features = cleanData[feature_list].join(dummy_ranks_PD.ix[:,:]).join(dummy_ranks_DAY.ix[:,:]).join(address_features.ix[:,:])
    print "creating new features"
    features["IsDup"]=pd.Series(features.duplicated()|features.duplicated(take_last=True)).apply(int)
    features["Awake"]=features["Time"].apply(lambda x: 1 if (x==0 or (x>=8 and x<=23)) else 0)
    features["Summer"], features["Fall"], features["Winter"], features["Spring"]=zip(*features["Month"].apply(get_season))
    if "Category" in df.columns:
        print df["Category"]
        labels = df["Category"].astype('category')
        #labels = df["Category"].unique()
#         label_names=labels.unique()
#         labels=labels.cat.rename_categories(range(len(label_names)))
    else:
        labels=None
    return features,labels

In [7]:
addresses=sorted(trainDF["Address"].unique())
categories=sorted(trainDF["Category"].unique())
C_counts=trainDF.groupby(["Category"]).size()
A_C_counts=trainDF.groupby(["Address","Category"]).size()
A_counts=trainDF.groupby(["Address"]).size()
logodds={}
logoddsPA={}
MIN_CAT_COUNTS=2
default_logodds=np.log(C_counts/len(trainDF))-np.log(1.0-C_counts/float(len(trainDF)))
for addr in addresses:
    PA=A_counts[addr]/float(len(trainDF))
    logoddsPA[addr]=np.log(PA)-np.log(1.-PA)
    logodds[addr]=deepcopy(default_logodds)
    for cat in A_C_counts[addr].keys():
        if (A_C_counts[addr][cat]>MIN_CAT_COUNTS) and A_C_counts[addr][cat]<A_counts[addr]:
            PA=A_C_counts[addr][cat]/float(A_counts[addr])
            logodds[addr][categories.index(cat)]=np.log(PA)-np.log(1.0-PA)
    logodds[addr]=pd.Series(logodds[addr])
    logodds[addr].index=range(len(categories))

In [9]:
features, labels=parse_data(trainDF,logodds,logoddsPA)

Creating address features
Parsing dates


/Library/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Creating one-hot variables
droping processed columns
joining one-hot features
creating new features


/Library/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:35: FutureWarning: the take_last=True keyword is deprecated, use keep='last' instead


0                       WARRANTS
1                 OTHER OFFENSES
2                 OTHER OFFENSES
3                  LARCENY/THEFT
4                  LARCENY/THEFT
5                  LARCENY/THEFT
6                  VEHICLE THEFT
7                  VEHICLE THEFT
8                  LARCENY/THEFT
9                  LARCENY/THEFT
10                 LARCENY/THEFT
11                OTHER OFFENSES
12                     VANDALISM
13                 LARCENY/THEFT
14                  NON-CRIMINAL
15                  NON-CRIMINAL
16                       ROBBERY
17                       ASSAULT
18                OTHER OFFENSES
19                  NON-CRIMINAL
20                 LARCENY/THEFT
21                       ROBBERY
22                      WARRANTS
23                  NON-CRIMINAL
24                 LARCENY/THEFT
25                  NON-CRIMINAL
26                 LARCENY/THEFT
27                 LARCENY/THEFT
28                 LARCENY/THEFT
29                OTHER OFFENSES
          

In [10]:
print features.columns.tolist()
print len(features.columns)

['X', 'Y', 'Time', 'Day', 'Month', 'Year', 'IsInterection', 'logoddsPA', 'PD_BAYVIEW', 'PD_CENTRAL', 'PD_INGLESIDE', 'PD_MISSION', 'PD_NORTHERN', 'PD_PARK', 'PD_RICHMOND', 'PD_SOUTHERN', 'PD_TARAVAL', 'PD_TENDERLOIN', 'DAY_Friday', 'DAY_Monday', 'DAY_Saturday', 'DAY_Sunday', 'DAY_Thursday', 'DAY_Tuesday', 'DAY_Wednesday', 'logodds0', 'logodds1', 'logodds2', 'logodds3', 'logodds4', 'logodds5', 'logodds6', 'logodds7', 'logodds8', 'logodds9', 'logodds10', 'logodds11', 'logodds12', 'logodds13', 'logodds14', 'logodds15', 'logodds16', 'logodds17', 'logodds18', 'logodds19', 'logodds20', 'logodds21', 'logodds22', 'logodds23', 'logodds24', 'logodds25', 'logodds26', 'logodds27', 'logodds28', 'logodds29', 'logodds30', 'logodds31', 'logodds32', 'logodds33', 'logodds34', 'logodds35', 'logodds36', 'logodds37', 'logodds38', 'IsDup', 'Awake', 'Summer', 'Fall', 'Winter', 'Spring']
70


In [11]:
# num_feature_list=["Time","Day","Month","Year","DayOfWeek"]
collist=features.columns.tolist()
scaler = preprocessing.StandardScaler()
scaler.fit(features)
features[collist]=scaler.transform(features)

In [12]:
new_PCA=PCA(n_components=60)
new_PCA.fit(features)
plt.plot(new_PCA.explained_variance_ratio_)
plt.yscale('log')
plt.title("PCA explained ratio of features")
print new_PCA.explained_variance_ratio_

[ 0.07714363  0.06267647  0.04861061  0.04300871  0.03340587  0.03085896
  0.02834616  0.02731142  0.02509795  0.02393235  0.02272111  0.02029053
  0.02000664  0.019452    0.01918369  0.01820193  0.01729548  0.01724542
  0.01715838  0.01714426  0.01703779  0.01695665  0.01668107  0.01651773
  0.01626539  0.01588035  0.01489293  0.01472615  0.01422875  0.01412131
  0.0136947   0.01330571  0.01234474  0.01181851  0.01158515  0.01126484
  0.01104143  0.0105683   0.01032344  0.00943963  0.00903894  0.00838257
  0.00819149  0.00795652  0.0074206   0.00726109  0.00707183  0.00675531
  0.0067189   0.00646287  0.00634686  0.00612685  0.0060737   0.00556134
  0.00526841  0.00516993  0.0048638   0.00452517  0.00417899  0.00407036]


In [13]:
plt.plot(new_PCA.explained_variance_ratio_.cumsum())
plt.title("cumsum of PCA explained ratio")

In [13]:
sss = StratifiedShuffleSplit(labels, train_size=0.5)
for train_index, test_index in sss:
    features_train,features_test=features.iloc[train_index],features.iloc[test_index]
    labels_train,labels_test=labels[train_index],labels[test_index]
features_test.index=range(len(features_test))
features_train.index=range(len(features_train))
labels_train.index=range(len(labels_train))
labels_test.index=range(len(labels_test))
features.index=range(len(features))
labels.index=range(len(labels))

In [14]:
def build_and_fit_model(X_train,y_train,X_test=None,y_test=None,hn=32,dp=0.5,layers=1,epochs=1,batches=64,verbose=0):
    input_dim=X_train.shape[1]
    output_dim=len(y_train.unique())
    Y_train=np_utils.to_categorical(y_train.cat.rename_categories(range(len(y_train.unique()))))
    print output_dim, input_dim
    model = Sequential()
    model.add(Dense(hn, input_shape=(input_dim,)))
    model.add(PReLU())
    model.add(Dropout(dp))

    for i in range(layers):
        model.add(Dense(hn))
        model.add(PReLU())
        model.add(BatchNormalization())
        model.add(Dropout(dp))

    model.add(Dense(output_dim))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    if X_test is not None:
        Y_test=np_utils.to_categorical(y_test.cat.rename_categories(range(len(y_test.unique()))))
        fitting=model.fit(X_train, Y_train, nb_epoch=epochs, batch_size=batches,verbose=verbose,validation_data=(X_test,Y_test))
        test_score = log_loss(y_test, model.predict_proba(X_test,verbose=0))
    else:
        model.fit(X_train, Y_train, nb_epoch=epochs, batch_size=batches,verbose=verbose)
        fitting=0
        test_score = 0
    return test_score, fitting, model

In [15]:
len(features.columns)

70

In [16]:
N_EPOCHS=20
N_HN=128
N_LAYERS=1
DP=0.5

In [17]:
score, fitting, model = build_and_fit_model(features_train.as_matrix(),labels_train,X_test=features_test.as_matrix(),y_test=labels_test,hn=N_HN,layers=N_LAYERS,epochs=N_EPOCHS,verbose=2,dp=DP)

39 70
Train on 438991 samples, validate on 438988 samples
Epoch 1/20
29s - loss: 2.4242 - val_loss: 2.2241
Epoch 2/20
28s - loss: 2.2882 - val_loss: 2.2121
Epoch 3/20
27s - loss: 2.2734 - val_loss: 2.2050
Epoch 4/20
30s - loss: 2.2665 - val_loss: 2.2027
Epoch 5/20
34s - loss: 2.2618 - val_loss: 2.1974
Epoch 6/20
34s - loss: 2.2593 - val_loss: 2.1967
Epoch 7/20
29s - loss: 2.2560 - val_loss: 2.1949
Epoch 8/20
29s - loss: 2.2536 - val_loss: 2.1934
Epoch 9/20
28s - loss: 2.2518 - val_loss: 2.1909
Epoch 10/20
28s - loss: 2.2490 - val_loss: 2.1892
Epoch 11/20
28s - loss: 2.2493 - val_loss: 2.1894
Epoch 12/20
29s - loss: 2.2486 - val_loss: 2.1864
Epoch 13/20
29s - loss: 2.2468 - val_loss: 2.1865
Epoch 14/20
29s - loss: 2.2454 - val_loss: 2.1877
Epoch 15/20
29s - loss: 2.2452 - val_loss: 2.1868
Epoch 16/20
28s - loss: 2.2443 - val_loss: 2.1870
Epoch 17/20
29s - loss: 2.2442 - val_loss: 2.1852
Epoch 18/20
29s - loss: 2.2437 - val_loss: 2.1848
Epoch 19/20
29s - loss: 2.2437 - val_loss: 2.1850
E

In [18]:
print "all", log_loss(labels, model.predict_proba(features.as_matrix(),verbose=0))
print "train", log_loss(labels_train, model.predict_proba(features_train.as_matrix(),verbose=0))
print "test", log_loss(labels_test, model.predict_proba(features_test.as_matrix(),verbose=0))

all 2.18099860518
train 2.177833624
test 2.18416160688


In [19]:

plt.plot(fitting.history['val_loss'],label="validation")
plt.plot(fitting.history['loss'],label="train")
# plt.xscale('log')
plt.legend()

In [20]:
score, fitting, model = build_and_fit_model(features.as_matrix(),labels,hn=N_HN,layers=N_LAYERS,epochs=N_EPOCHS,verbose=2,dp=DP)

39 70
Epoch 1/20
53s - loss: 2.3542
Epoch 2/20
57s - loss: 2.2704
Epoch 3/20
58s - loss: 2.2610
Epoch 4/20
52s - loss: 2.2547
Epoch 5/20
54s - loss: 2.2521
Epoch 6/20
53s - loss: 2.2491
Epoch 7/20
51s - loss: 2.2470
Epoch 8/20
52s - loss: 2.2460
Epoch 9/20
54s - loss: 2.2454
Epoch 10/20
55s - loss: 2.2432
Epoch 11/20
53s - loss: 2.2431
Epoch 12/20
55s - loss: 2.2419
Epoch 13/20
60s - loss: 2.2424
Epoch 14/20
55s - loss: 2.2406
Epoch 15/20
57s - loss: 2.2406
Epoch 16/20
50s - loss: 2.2406
Epoch 17/20
50s - loss: 2.2396
Epoch 18/20
49s - loss: 2.2394
Epoch 19/20
49s - loss: 2.2386
Epoch 20/20
49s - loss: 2.2381


In [21]:
print "all", log_loss(labels, model.predict_proba(features.as_matrix(),verbose=0))
print "train", log_loss(labels_train, model.predict_proba(features_train.as_matrix(),verbose=0))
print "test", log_loss(labels_test, model.predict_proba(features_test.as_matrix(),verbose=0))

all 2.1753961041
train 2.17610193679
test 2.17468864752


In [22]:
testDF=pd.read_csv("../data/test.csv")
testDF[["X","Y"]]=xy_scaler.transform(testDF[["X","Y"]])
#set outliers to 0
testDF["X"]=testDF["X"].apply(lambda x: 0 if abs(x)>5 else x)
testDF["Y"]=testDF["Y"].apply(lambda y: 0 if abs(y)>5 else y)

In [23]:
new_addresses=sorted(testDF["Address"].unique())
new_A_counts=testDF.groupby("Address").size()
only_new=set(new_addresses+addresses)-set(addresses)
only_old=set(new_addresses+addresses)-set(new_addresses)
in_both=set(new_addresses).intersection(addresses)
for addr in only_new:
    PA=new_A_counts[addr]/float(len(testDF)+len(trainDF))
    logoddsPA[addr]=np.log(PA)-np.log(1.-PA)
    logodds[addr]=deepcopy(default_logodds)
    logodds[addr].index=range(len(categories))
for addr in in_both:
    PA=(A_counts[addr]+new_A_counts[addr])/float(len(testDF)+len(trainDF))
    logoddsPA[addr]=np.log(PA)-np.log(1.-PA)

In [ ]:
features_sub, _=parse_data(testDF,logodds,logoddsPA)
# scaler.fit(features_test)

Creating address features
Parsing dates


In [ ]:
collist=features_sub.columns.tolist()
print collist

In [ ]:
features_sub[collist]=scaler.transform(features_sub[collist])

predDF=pd.DataFrame(model.predict_proba(features_sub.as_matrix(),verbose=0),columns=sorted(labels.unique()))
predDF.head()

In [ ]:
predDF.to_csv("Crime_SF_NN_Logodds.csv",index_label="Id",na_rep="0")